# StyleGrid I

Extending the [DataGrid](./DataGrid.ipynb), `StyleGrid` adds support for top-level styles of the rendered grid. Advanced customization is possible with [custom renderers](./StyleGrid%20II.ipynb).

In [ ]:
from wxyz.datagrid import StyleGrid, GridStyle

In [ ]:
import pandas as pd, traitlets as T, ipywidgets as W, IPython.display as D
import importnb
from wxyz.html import AlphaColorPicker 
from wxyz.lab import DockBox

In [ ]:
with importnb.Notebook():
    try:
        from DataGrid import make_grid
    except:
        from .DataGrid import make_grid

## GridStyle

In [ ]:
def make_a_grid_styler(style=None, grid=None, pattern='color'):
    special = ['columnBackgroundColor', 'rowBackgroundColor']

    style = style or GridStyle()
    colors = [
        AlphaColorPicker(description=n) 
        for n in style.trait_names() 
        if pattern in n.lower()
        and n not in special
    ]
    
    box = W.VBox(colors, _dom_classes=["my-style"], layout=dict(display="flex", flex_flow="row wrap"))
    box.children += (W.HTML("""<style>
        .my-style .widget-colorpicker {
            display: flex;
            flex-direction: column;
            height: 4em;
        }
        .my-style .widget-colorpicker label {
            display: flex;
            width: 100%;
        }
    </style>"""),)
    
    if grid:
        [T.dlink((c, 'value'), (grid.grid_style, c.description)) for c in colors]
    
    return box

In [ ]:
if __name__ == "__main__":
    styler = make_a_grid_styler()
    D.display(styler)

## Make a Grid

In [ ]:
_default_container = lambda *args, **kwargs: [
    kwargs.update(layout=dict(height="60vh"), dock_layout={'type': 'split-area',
 'orientation': 'horizontal',
 'children': [{'type': 'tab-area', 'widgets': [0], 'currentIndex': 0},
  {'type': 'tab-area', 'widgets': [1], 'currentIndex': 0}],
 'sizes': [8, 3]}),
    DockBox(*args, **kwargs)
][-1]

def make_style_grid_i(GridClass=StyleGrid, df=None, container=None):
    container = container or _default_container
    pg = make_grid(GridClass, df=df)
    dg = pg.dg
    st = make_a_grid_styler(grid=dg)

    rs = W.IntSlider(description='row height')
    cs = W.IntSlider(description='column width')
    hv = W.SelectionSlider(description="headers?", default_value="all", options=["all", "row", "column", "none"])
    hr = W.HTML()
    hc = W.HTML()

    W.jslink((dg, "row_size"), (rs, "value"))
    W.jslink((dg, "column_size"), (cs, "value"))
    W.link((dg, "header_visibility"), (hv, "value"))
    grid_playground = container([
        W.VBox([
            pg.children[1], 
        ], layout=dict(flex="1")),
        W.VBox([*pg.children[0].children, hr, hc, cs, rs, hv, *st.children]), 
    ])
    grid_playground.dg = dg
    return grid_playground

In [ ]:
if __name__ == "__main__":
    pg = make_style_grid_i()
    display(pg)